In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

In [2]:
mimic_data = pd.read_csv("data/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
binary_data = mimic_data[['text','No Finding']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,No Finding
0,"b"" final repor...",0.0
1,b' final repor...,0.0
2,b' final repor...,1.0
3,b' final repor...,1.0
4,b' final repor...,1.0


In [4]:
binary_data.groupby('No Finding').count()

,text
No Finding,
0.0,152372
1.0,75455


In [5]:
binary_data.text[0]

'b"                                 final report\\n type of examination:  chest pa and lateral.\\n \\n indication:  ___-year-old male patient with recent pneumonia diagnosed and\\n treated at another facility.  x-ray not available, now with continued cough\\n and wheeze, history of copd, remaining evidence of pneumonia?\\n \\n findings:  pa and lateral chest views were obtained with patient in upright\\n position.  analysis is performed in direct comparison with the next preceding\\n chest examination of ___.  the heart size remains normal.  no\\n typical configurational abnormality is seen.  the thoracic aorta is moderately\\n widened and somewhat elongated but no local contour abnormalities are\\n identified.  the pulmonary vasculature is not congested.  there exists,\\n however, some irregular peripheral vascular distribution most marked on the\\n bases and coinciding with some slightly hypertranslucent pulmonary areas and\\n flattened low positioned diaphragms are indicative of cop

In [6]:
y = binary_data['No Finding']
X = binary_data.drop(columns=['No Finding'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [7]:
X_train.head()

,text
0,b' final repor...
1,"b"" wet read: ___ ___ 9:56 pm\n new left pleur..."
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [23]:
X_train.text[0]

"b'                                 final report\\n indication:  ___-year-old female with cough and fever.  please perform a\\n lateral view to evaluate for pneumonia.\\n \\n comparison:  frontal chest radiograph performed approximately one hour prior\\n to this exam.\\n \\n technique:  a leftward rotated ap view and a lateral view of the chest were\\n obtained.\\n \\n findings:  the frontal view is extremely rotated to the left, with complete\\n projection of the mediastinum over the left lung, which limits assessment. \\n the expanded right lung is unremarkable.  assessment in the lateral view is\\n also limited due to superimposition of the arms, but allowing for technical\\n limitations, there is no spine sign, pleural effusion, or abnormality in the\\n anterior mediastinum. no pneumothorax is identified.  artifacts from external\\n hair devices are again seen.\\n \\n impression:  limited examination.  no evidence of acute cardiopulmonary\\n process.\\n'"

In [24]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [25]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0].lower())))

'b final report type of examination chest pa and lateral indication yearold male patient with recent pneumonia diagnosed and treated at another facility xray not available now with continued cough and wheeze history of copd remaining evidence of pneumonia findings pa and lateral chest views were obtained with patient in upright position analysis is performed in direct comparison with the next preceding chest examination of the heart size remains normal no typical configurational abnormality is seen the thoracic aorta is moderately widened and somewhat elongated but no local contour abnormalities are identified the pulmonary vasculature is not congested there exists however some irregular peripheral vascular distribution most marked on the bases and coinciding with some slightly hypertranslucent pulmonary areas and flattened low positioned diaphragms are indicative of copd when direct comparison is made with the previous examination of there is a hazy mild degree of density in the left 

In [26]:
spacy_tok(binary_data.text[0])

['b',
 'final',
 'report',
 'type',
 'of',
 'examination',
 'chest',
 'pa',
 'and',
 'lateral',
 'indication',
 'yearold',
 'male',
 'patient',
 'with',
 'recent',
 'pneumonia',
 'diagnosed',
 'and',
 'treated',
 'at',
 'another',
 'facility',
 'xray',
 'not',
 'available',
 'now',
 'with',
 'continued',
 'cough',
 'and',
 'wheeze',
 'history',
 'of',
 'copd',
 'remaining',
 'evidence',
 'of',
 'pneumonia',
 'findings',
 'pa',
 'and',
 'lateral',
 'chest',
 'views',
 'were',
 'obtained',
 'with',
 'patient',
 'in',
 'upright',
 'position',
 'analysis',
 'is',
 'performed',
 'in',
 'direct',
 'comparison',
 'with',
 'the',
 'next',
 'preceding',
 'chest',
 'examination',
 'of',
 'the',
 'heart',
 'size',
 'remains',
 'normal',
 'no',
 'typical',
 'configurational',
 'abnormality',
 'is',
 'seen',
 'the',
 'thoracic',
 'aorta',
 'is',
 'moderately',
 'widened',
 'and',
 'somewhat',
 'elongated',
 'but',
 'no',
 'local',
 'contour',
 'abnormalities',
 'are',
 'identified',
 'the',
 'pulmo

In [27]:
binary_train = binary_data.copy()
binary_train.head()

,text,No Finding
0,"b"" final repor...",0.0
1,b' final repor...,0.0
2,b' final repor...,1.0
3,b' final repor...,1.0
4,b' final repor...,1.0


In [32]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word))
    return counts

In [33]:
word_count = get_counts(binary_train.text)

In [34]:
len(word_count.keys())

37093

In [35]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [36]:
len(word_count.keys())

15527

In [37]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [38]:
len(words)

15529

In [39]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [40]:
encode_sentence(binary_train.text[0])

array([  1,   3,   1,   5,   6,   1,   8,   9,  10,   1,   1,   1,   1,
        14,  15,  16,  17,  18,  19,   1,  20,  21,  22,   1,   1,  25,
         1,  27,  16,  28,   1,  10,   1,  31,   6,   1,  33,  34,   6,
         1,   1,   1,   9,  10,  11,   8,  36,  37,  38,  16,  15,  39,
         1,   1,  42,  43,  44,  39,  45,  46,  16,  47,  48,   1,   8,
         7,   6,   1,  47,  50,  51,  52,   1,   1,  55,  56,  57,  43,
         1,  47,  59,  60,  43,   1,  62,  10,  63,  64,  65,  54,  66,
        67,  68,   1,   1,  47,  71,  72,  43,  25,   1,  74,   1,   1,
        77,  78,  79,  80,  81,  82,  83,  84,   1,  85,  10,  86,  16,
        77,  87,  88,  71,  89,   1,  90,  91,  92,  93,  69,  94,   6,
         1,  95,   1,  46,  43,  96,  16,  47,  97,   7,   6,   1,  74,
        43,  98,  99,   1, 101,   6, 102,  39,  47, 103, 104, 105,  98,
       106,  66, 107,   1,  47, 108, 109,   6,  47, 110, 111,  84,  47,
       103,   1, 113, 114,   1, 115,  16,  77, 116, 117, 118, 11

## Dataset and data loaders

In [41]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [42]:
binary_train = Binary_Mimic(X_train, y_train, vocab2index)
binary_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [43]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [44]:
data = [binary_train[0], binary_train[1], binary_train[2]]
data

[(array([   1,    3,    1,    1,    1,  396,   16,   29,   10,    1,  528,
         4214,    1,   11,  475,  166,  190,  191,    1,    1,    1,  174,
            8,  175,   44, 1031,  427, 1776,    1,  166,  113,    1,    1,
            1,   98, 1616,  739,  194,  475,   10,   98,   11,  475,    6,
           47,    8,    1,    1,    1,    1,   47,  174,  475,   43, 2208,
          739,  166,   47,    1,   16,    1, 1310,    6,   47,  677,  411,
           47,  103,    1,  124,  761,    1,    1,   47,  493,  169,  239,
           43,    1, 1067,   39,   47,   11,  475,    1,  675,  694,  283,
          166, 2084,    6,   47,    1,   65,  330,  191,    1,    1,   74,
           43,   54,  130,    1,  140,    1,  154,   57,   39,    1,  129,
            1,   54,  204,   43,    1, 4226,  183,    1, 5668,  296,   69,
          355,    1,    1,    1,  694,    1,   54,   34,    6,  192,    1,
            1]), 1.0),
 (array([   1,  284,    1,    1,    1,    1,    1,  135,  103,  140,    1,
  

In [45]:
collate_fn(data)

(tensor([[   1,  284,    1,    1,    1,    1,    1,  135,  103,  140,    1,  483,
           214,    1,  254,    1,  810,  102,    1,  294,  328,    1,  490,   16,
             1,    1,  126,  717,   21,    1,   84,    1,   21,  113,    1,    8,
           828,  448,    1,    1,    3,    1,  232,  194,   40,    8,  353,    1,
            21,    1,    1,    1,  145,    1,    1,   16,  281,    1,  591,    1,
             1,   46,   43,   96,  166,   47,    1,  302,  303,    6,    1,   21,
             1,    1,  513,  232,  194,   40,    8,  353,    1,   21,    1,   43,
             1,    1,    1,    1,    1,  297,   91,  239,  373,   16,  630,  388,
            21,   47,  103,  104,  124,    1,  444,    1,  680,  495,  154,   18,
             1,  254,    1,    1,    1,   95,  251,  166,   47,  302,  303,   74,
            43,   54,    1,  361,    1,   54,   34,    6,   71,    1,   54,    1,
             1,  273,   10,  198,  200,   69,  132,  421,   91,  239,  373,    1,
           331, 

In [46]:
train_loader = DataLoader(binary_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
sents, lengths, labels = next(iter(train_loader))
sents

tensor([[   1,    3,    1,  232,    8,    1,   16,   46,    1,    1,    1,    1,
          361, 1038,   10, 1084,    6,  452,    1,    1,  434,    6,  169,  229,
          198,    1,  377,  283,  166,   98,  811,    1,   80, 2420,   10,  240,
          198,  681,   84,   17,  245,    1,    1,  132,  180,  666,  126,   71,
            1,  515,   10,  100,    1,  297,  299,  140,  271,    1,  438,  581,
          239,  280,  124,  443,  444,  551,    1,  393,    1,  321, 2676,    6,
         1078,  109,    6,  229,    1,  684,   47,  932,    6,    1],
        [   1,    3,    1,    1,    8,    1,   10,    1,    1,    1,    1,  157,
          158,  159,    1, 1678,    1,  642,  164,  642,    1,    1,    1,    8,
          195,    1,  536,    1,    1,    1,    1,    1,  163,  299, 1209, 1550,
          610,  861,  258,    1,    1,  550,  551,  133,    1,  163,  299,  140,
          271,    1,    1,  225,  226,   43,   98,   53, 1210,    1,   74,   43,
           54,   71,    1,    1,    0, 

In [47]:
sents.shape, labels.shape

(torch.Size([2, 82]), torch.Size([2]))

In [48]:
lengths

[82, 64]

## Training

In [53]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long()#.cuda()
            y = y.float()#.cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))

In [54]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long()#.cuda()
        y = y.float().unsqueeze(1)#.cuda()
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [55]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [56]:
batch_size = 50000
train_dl = DataLoader(binary_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(binary_valid, batch_size=batch_size, collate_fn=collate_fn)

In [57]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50)#.cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

15529


In [ ]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=1)

torch.Size([50000, 477])


## Use Glove to create pre-trained embedding 

In [69]:
def loadGloveModel(gloveFile='data/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [70]:
word_vecs = loadGloveModel()

In [80]:
word_count = get_counts(binary_train.text)

In [81]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 37093


In [82]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [83]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

20222


In [84]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [85]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [86]:
len(pretrained_weight)

20224

In [87]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1397,  0.0364, -0.0639,  ...,  0.1270,  0.2325,  0.1952],
        [-0.3012,  0.3320,  0.1582,  ..., -0.7797,  0.9873,  0.6246],
        ...,
        [-0.1321, -0.3749, -0.5517,  ...,  0.9329,  0.3362, -0.2008],
        [ 0.8547,  0.1916,  0.0963,  ...,  0.1707, -0.2616,  0.5350],
        [-0.0111, -1.1496, -0.5092,  ...,  0.0186, -0.9912,  0.1102]])